In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [9]:
STORM_NUMBER = 2
TITLE = "19 October 2020"
SUBTITLE = "4 PM 19 October 2020"
sts = utc(2020, 10, 19, 17)
ets = utc(2020, 10, 19, 23, 0)
# Get available data
ctx = {'coop': 'no', 'csector': 'IA', 't': 'state', 'sz': 15, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [10]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2019-2020 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [13]:
cull = []
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
hardcode = [
    ['Z58', 3],
    ['Z59', 2.],
    ['Z72', 3.],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=0.0001)
res = mp.postprocess(filename='201019.png')
mp.close()

    state  wfo  val        lon        lat                             geo  \
61      Z  NaN  0.0 -97.109778  40.201616  POINT (245614.744 -529110.167)   
62      Z  NaN  0.0 -96.580703  40.186179  POINT (290614.744 -529110.167)   
63      Z  NaN  0.0 -96.051856  40.168154  POINT (335614.744 -529110.167)   
64      Z  NaN  0.0 -95.523271  40.147545  POINT (380614.744 -529110.167)   
65      Z  NaN  0.0 -94.994984  40.124354  POINT (425614.744 -529110.167)   
..    ...  ...  ...        ...        ...                             ...   
183     Z  NaN  0.0 -91.905237  43.592862  POINT (650614.744 -124110.167)   
184     Z  NaN  0.0 -91.348846  43.551335  POINT (695614.744 -124110.167)   
185     Z  NaN  0.0 -90.793145  43.507064  POINT (740614.744 -124110.167)   
186     Z  NaN  0.0 -90.238177  43.460054  POINT (785614.744 -124110.167)   
187     Z  NaN  0.0 -89.683981  43.410311  POINT (830614.744 -124110.167)   

     used_for_analysis nwsli  plotme source  
61                True    Z1 